In [ ]:
!pip install scikit-learn tensorflow pandas datasets peft evaluate accelerate>=0.20.1 --extra-index-url=https://pypi.nvidia.com cudf-cu11

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.utils import compute_class_weight
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import tensorflow as tf
import pandas as pd
import numpy as np

from datasets import load_dataset, DatasetDict, Dataset

import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm


from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:
## euphemistic test sentence
text = "He was between jobs last month"

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline


### Need to incorporate loading tokenizer to model
## Simply push_to_hub the tokenizer that i fine-tuned previously to have access to config and model
pipe = pipeline("text-classification", model="nhankins/zh_euph_classifier_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
pipe(text)

[{'label': 'Euphemistic', 'score': 0.9081451892852783}]

In [ ]:
### Mount gdrive directory where test data is

#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
## make test data into dataframe
testing_df = pd.read_csv('/content/drive/MyDrive/YO_test2.csv', usecols=['text'])


In [ ]:
testing_df

,text
0,Mo ma [PET_BOUNDARY]La[PET_BOUNDARY] e bi eni ...
1,e fi ahon yin yen le ibi ido obinrin yen ki e ...
2,Iru awọn ironu yii le mu ọ ni [PET_BOUNDARY]It...
3,"Gẹgẹ bi iroyin to n lọ, awọn ololufẹ mejeeji n..."
4,[PET_BOUNDARY]Àkàrà tu sépo[PET_BOUNDARY] ! ob...
...,...
664,"Ejo eyin ara group yii, ta lo fe ka jo ma bara..."
665,Awon obinrin kan n lọ fun iṣẹ [PET_BOUNDARY]Ab...
666,Odara ki obirin fa irun [PET_BOUNDARY]Abe[PET_...
667,"OKUNRIN TO TETE RELEASE LORI OBINRIN, ATI [PET..."


In [ ]:
## Make dataframe into Hugging Face compatible dataset
ds = Dataset.from_pandas(testing_df)


In [ ]:
## Loop through all examples in test data

for out in tqdm(pipe(KeyDataset(ds, "text"))):
    print(out)

  0%|          | 0/669 [00:00<?, ?it/s]

{'label': 'Euphemistic', 'score': 0.5824960470199585}
{'label': 'Euphemistic', 'score': 0.5851101875305176}
{'label': 'Euphemistic', 'score': 0.6315162181854248}
{'label': 'Euphemistic', 'score': 0.5093621015548706}
{'label': 'Euphemistic', 'score': 0.6540184617042542}
{'label': 'Euphemistic', 'score': 0.6422475576400757}
{'label': 'Euphemistic', 'score': 0.6255802512168884}
{'label': 'Euphemistic', 'score': 0.6648196578025818}
{'label': 'Euphemistic', 'score': 0.6257359385490417}
{'label': 'Euphemistic', 'score': 0.5253362059593201}
{'label': 'Euphemistic', 'score': 0.5970354080200195}
{'label': 'Euphemistic', 'score': 0.5553457140922546}
{'label': 'Euphemistic', 'score': 0.6986387968063354}
{'label': 'Euphemistic', 'score': 0.6474447846412659}
{'label': 'Euphemistic', 'score': 0.639011561870575}
{'label': 'Euphemistic', 'score': 0.7043970227241516}
{'label': 'Euphemistic', 'score': 0.6392748355865479}
{'label': 'Euphemistic', 'score': 0.5755575299263}
{'label': 'Euphemistic', 'score'

In [ ]:
#For chinese
## Need parameter adjustments due to tokenization differences
for out in tqdm(pipe(KeyDataset(ds, "text"), padding= True, truncation=True, max_length= 512, add_special_tokens = True)):
    print(out)

  0%|          | 0/1226 [00:00<?, ?it/s]

{'label': 'Euphemistic', 'score': 0.6315255165100098}
{'label': 'Euphemistic', 'score': 0.7595738768577576}
{'label': 'Euphemistic', 'score': 0.7309468984603882}
{'label': 'Euphemistic', 'score': 0.5520365834236145}
{'label': 'Euphemistic', 'score': 0.6619324684143066}
{'label': 'Euphemistic', 'score': 0.7854775190353394}
{'label': 'Euphemistic', 'score': 0.7926096320152283}
{'label': 'Euphemistic', 'score': 0.7420020699501038}
{'label': 'Euphemistic', 'score': 0.7593699097633362}
{'label': 'Euphemistic', 'score': 0.7569578289985657}
{'label': 'Euphemistic', 'score': 0.5255153179168701}
{'label': 'Euphemistic', 'score': 0.6589422225952148}
{'label': 'Euphemistic', 'score': 0.7207984924316406}
{'label': 'Euphemistic', 'score': 0.7402671575546265}
{'label': 'Euphemistic', 'score': 0.541149377822876}
{'label': 'Euphemistic', 'score': 0.7668119072914124}
{'label': 'Euphemistic', 'score': 0.6726176738739014}
{'label': 'Euphemistic', 'score': 0.7190114259719849}
{'label': 'Euphemistic', 'sco

In [ ]:
###Rough draft of getting the inference to work


#Trying to get indexes

# output = tqdm(pipe(KeyDataset(ds, "text"), padding= True, truncation=True, max_length= 512, add_special_tokens = True))

# for idx, extracted_entities in enumerate(output):
#     print("Original text:\n{}".format(ds[idx]["text"]))
#     print("Extracted entities:")
#     for entity in extracted_entities:
#         print(entity)